In [ ]:
# build a travel-planning agent 
# tavily-python: API for AI web search
# openai: API for GPT calls

#!pip install requests tavily-python openai


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [102]:
# system prompt for agent

AGENT_SYSTEM_PROMPT = """
你是一个智能旅行助手。你的任务是分析用户的请求，并使用可用工具一步步地解决问题。

# 可用工具:
- `get_weather(city: str)`: 查询指定城市的实时天气。
- `get_attraction(city: str, weather: str)`: 根据城市和天气搜索推荐的旅游景点。

# 输出格式要求:
你的每次回复必须严格遵循以下格式，包含一对Thought和Action：

Thought: [你的思考过程和下一步计划]
Action: [你要执行的具体行动]

Action的格式必须是以下之一：
1. 调用工具：function_name(arg_name="arg_value")
2. 结束任务：Finish[最终答案]

# 重要提示:
- 每次只输出一对Thought-Action
- Action必须在同一行，不要换行
- 当收集到足够信息可以回答用户问题时，必须使用 Action: Finish[最终答案] 格式结束

请开始吧！
"""


In [103]:
# define tool_1 for weather search

import requests

def get_weather(city: str) -> str:
    """
    call wttr.in API for real weather information
    """
    
    # API node, call for JSON data
    url=f"https://wttr.in/{city}?format=j1"

    try:
        # send an http request
        response = requests.get(url)
        # check response state code is 200(success) or not
        response.raise_for_status()
        #get returned JSON data
        data = response.json()

        # parse current weather information
        current_condition = data['current_condition'][0]
        weather_desc = current_condition['weatherDesc'][0]['value']
        temp_c = current_condition['temp_C']

        # change format as natural language and return
        return f"{city}当前天气：{weather_desc}，气温{temp_c}摄氏度"
    
    except requests.exceptions.RequestException as e:
        # handle network error
        return f"错误：查询天气时遇到网络问题-{e}"
    except (KeyError, IndexError) as e:
        # handle data parsing error
        return f"错误：解析天气数据失败，可能是城市名称无效-{e}"


In [104]:
# define tool_2 for tourist attraction web search

import os
from tavily import TavilyClient

def get_attraction(city: str, weather: str) -> str:
    """
    based on city and weather, search through Tavily Search API
    and return tourist attraction recommendation
    """

    # read API key from environment variable
    api_key = os.environ.get("TAVILY_API_KEY")
    if not api_key:
        return "错误：未配置TAVILY_API_KEY环境变量"
    
    # initialize Tavily client
    tavily = TavilyClient(api_key=api_key)

    # construct a query
    query = f"'{city}'在'{weather}'天气下最值得去的旅游景点推荐及理由"

    try:
        # call API, include_answer=True will return a general answer
        response = tavily.search(query=query, search_depth="basic", include_answer=True)

        # if Tavily returns a clean result to use directly
        # response['answer'] is a general answer based on all search results
        if response.get("answer"):
            return response["answer"]
        
        # if no general answer, format original result
        formatted_results = []
        for result in response.get("results", []):
            formatted_results.append(f"- {result['title']}: {result['content']}")


        if not formatted_results:
            return "抱歉，没有找到相关的旅游景点推荐。"
        
        return "根据搜索，为您找到一下信息:\n" + "\n".join(formatted_results)
    
    except Exception as e:
        return f"错误：执行Tavily搜索时出现问题-{e}"


In [105]:
# integrate all tool functions into a dictionary for future calls
available_tools = {
    "get_weather": get_weather,
    "get_attraction": get_attraction,
}


In [106]:
# design a openai-API-compatible client to call LLMs

from openai import OpenAI

class OpenAICompatibleClient:
    """
    a client to call any LLM service that is compatible with OpenAI API
    """
    def __init__(self, model: str, api_key: str, base_url: str):
        self.model = model
        self.client = OpenAI(api_key=api_key, base_url=base_url)

    def generate(self, prompt: str, system_prompt: str) -> str:
        """
        call LLM API to generate response
        """
        print("正在调用大语言模型...")
        try:
            messages = [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': prompt}
            ]
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                stream=False
            )
            answer = response.choices[0].message.content
            print("大语言模型响应成功。")
            return answer
        except Exception as e:
            print(f"调用LLM API时发生错误：{e}")
            return "错误：调用大语言模型服务时出错。"

In [107]:
# sign in ollama 
#!ollama signin
#!ollama pull qwen3:8b
#!pip install ollama

In [108]:
# integrate all components and execute planning with prompt-inserted LLMs

import re

# configurate LLM client
# based on your service, replace "YOUR_XXX" with related api keys, url and model id
API_KEY = "sk-Vtue8QmtLrzwDsDL22E94e4aA0C243F3Be39Ae9b6844AfB4"
BASE_URL = "https://aihubmix.com/v1"
MODEL_ID = "glm-4.7-flash-free"
TAVILY_API_KEY = "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"
os.environ['TAVILY_API_KEY'] = "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"

llm = OpenAICompatibleClient(
    model=MODEL_ID,
    api_key=API_KEY,
    base_url=BASE_URL
)

# initialization
user_prompt = "你好，查询一下今天北京的天气，然后根据天气推荐一个合适的旅游景点"
prompt_history = [f"用户请求：{user_prompt}"]

print(f"用户输入：{user_prompt}\n" + "="*40)

# execute main loop
for i in range(5): # max loop counts
    print(f"--- 循环 {i+1} ---\n")

    # build a prompt
    full_prompt = "\n".join(prompt_history)

    # call LLM for reasoning
    llm_output = llm.generate(full_prompt, system_prompt=AGENT_SYSTEM_PROMPT)
    # LLM may output redundant Thought-Action, which needs to be truncated
    match = re.search(r'(Thought:.*?Action:.*?)(?=\n\s*(?:Thought:|Action:|Observation:)|\Z)', llm_output, re.DOTALL)
    if match:
        truncated = match.group(1).strip()
        if truncated != llm_output.strip():
            llm_output = truncated
            print("已截断多余的 Thought-Action 对")
    print(f"模型输出：\n{llm_output}\n")
    prompt_history.append(llm_output)

    # parse and perform action
    action_match = re.search(r"Action:(.*)", llm_output, re.DOTALL)
    if not action_match:
        observation = "错误：未能解析到 Action 字段。请确保你的回复严格遵循 'Thought: ... Action: ...'的格式。"
        observation_str = f"Observation: {observation}"
        print(f"{observation_str}\n" + "="*40)
        prompt_history.append(observation_str)
        continue
    action_str = action_match.group(1).strip()

    if action_str.startswith("Finish"):
        final_answer = re.match(r"Finish\[(.*)\]", action_str)
        print(f"任务完成，最终答案：{final_answer}")
        break

    tool_name = re.search(r"(\w+)\(", action_str).group(1)
    args_str = re.search(r"\((.*)\)", action_str).group(1)
    kwargs = dict(re.findall(r'(\w+)="([^"]*)"', args_str))

    if tool_name in available_tools:
        observation = available_tools[tool_name](**kwargs)
    else:
        observation = f"错误：未定义的工具'{tool_name}'"

    # Record observation results
    observation_str = f"Observation:{observation}"
    print(f"{observation_str}\n" + "="*40)
    prompt_history.append(observation_str)

用户输入：你好，查询一下今天北京的天气，然后根据天气推荐一个合适的旅游景点
--- 循环 1 ---

正在调用大语言模型...
大语言模型响应成功。
模型输出：
Thought: 用户想要查询北京今天的天气，然后根据天气推荐景点。我需要先调用get_weather函数获取北京的实时天气信息，然后再根据天气情况调用get_attraction函数推荐合适的景点。

Action: get_weather(city="北京")

Observation:错误：查询天气时遇到网络问题-HTTPSConnectionPool(host='wttr.in', port=443): Max retries exceeded with url: /%E5%8C%97%E4%BA%AC?format=j1 (Caused by ConnectTimeoutError(<HTTPSConnection(host='wttr.in', port=443) at 0x1ab559f4590>, 'Connection to wttr.in timed out. (connect timeout=None)'))
--- 循环 2 ---

正在调用大语言模型...
大语言模型响应成功。
模型输出：
Thought: 上次查询天气遇到网络超时，现在重新尝试调用天气查询功能获取北京的实时天气信息。

Action: get_weather(city="北京")

Observation:北京当前天气：Sunny，气温12摄氏度
--- 循环 3 ---

正在调用大语言模型...
大语言模型响应成功。
模型输出：
Thought: 已成功获取北京今天的天气信息：天气晴朗，气温12摄氏度。现在我需要根据这个天气情况调用get_attraction函数来搜索适合的旅游景点推荐。

Action: get_attraction(city="北京", weather="Sunny")

Observation:在阳光明媚的北京天气下，故宫和颐和园是最值得去的旅游景点，前者展示历史建筑，后者有美丽的园林。
--- 循环 4 ---

正在调用大语言模型...
大语言模型响应成功。
模型输出：
Thought: 已成功获取北京今天的天气信息（Sunny，气温12摄氏度），并根据天气推荐了合适的旅游景点